*   **Heitor Rodrigues Sabino** -  202120101
*   **Ranulfo Mascari Neto** - 20212





---



## **1. Definições do Problema**

#### **Variáveis:**



- $ x_j $: quantidade (em gramas) do alimento $ j $ na dieta.


#### **Parâmetros:**



- $ a_{ij} $: quantidade do nutriente $ i $ presente no alimento $ j $ (em % do VDR).
- $ r_i $: valor diário recomendado para o nutriente $ i $ ($ r_i = 100 \% $).
- $ c_j $: custo do alimento $ j $.
- $ f_{\text{min}, j} $: limite inferior de consumo do alimento $ j $ ($ NA \to 0 $).
- $ f_{\text{max}, j} $: limite superior de consumo do alimento $ j $ ($ NA \to \infty $).

### **Objetivo:**

- Minimizar o custo total da dieta:
$
\text{Minimizar } \sum_{j=1}^n c_j \cdot x_j
$

### **Restrições:**

1. Garantir que cada nutriente $ i $ atenda ao valor diário recomendado:
$
\sum_{j=1}^n a_{ij} \cdot x_j \geq r_i, \quad \forall i = 1, \dots, m
$

2. Limitar as quantidades de cada alimento aos valores permitidos:
$
f_{\text{min}, j} \leq x_j \leq f_{\text{max}, j}, \quad \forall j = 1, \dots, n
$

## **2. Implementação**

### ***pip* e importações**

In [1]:
!pip install gurobipy

In [2]:
from gurobipy import Model, GRB
import pandas as pd

### **Dados**


#### **Alimentando base de dados**

In [3]:
df1 = pd.read_csv("McDonalds-amnt.wsv", sep='\s+',  comment='#')
df2 = pd.read_csv("McDonalds-food.wsv", sep='\s+',  comment='#')
nutrientes = pd.read_csv("McDonalds-nutr.wsv", sep='\s+',  comment='#')

In [4]:
alimentos = pd.merge(df1, df2, on="FOOD", how="inner")  # Tabela 1 e Tabela 2 combinadas

#### **Tratando**

In [5]:
# Preencher valores padrões para NA
alimentos['f_min'] = alimentos['f_min'].fillna(0)
alimentos['f_max'] = alimentos['f_max'].fillna(float('inf'))
alimentos.head()

,FOOD,Energia,Carb.,Prot.,G. Total,G. Sat.,Colest.,Fibra,Sódio,Cálcio,Ferro,cost,f_min,f_max,veg
0,Big Mac,25,14,33,49,55,18,14,40,19,46,6.9,0.0,inf,False
1,Big Tasty,42,15,55,100,109,35,20,63,38,59,9.4,0.0,inf,False
2,Quarterão com Queijo,28,12,41,58,73,29,14,51,28,71,6.4,0.0,inf,False
3,McNífico Bacon,31,13,45,67,73,31,16,52,21,79,7.9,0.0,inf,False
4,Cheddar McMelt,25,11,39,53,64,27,12,34,20,71,6.9,0.0,inf,False


#### **Extrair parâmetros**

In [15]:
a_ij = alimentos.iloc[:, 1:len(nutrientes)+1].values.T  # Quantidades de nutrientes nos alimentos
c_j = alimentos['cost'].values  # Custos dos alimentos
f_min = alimentos['f_min'].values  # Limites inferiores
f_max = alimentos['f_max'].values  # Limites superiores
r_i = nutrientes['n_min'].values  # VDR dos nutrientes

### **Modelo**

In [16]:
model = Model("DietaÓtima")

#### **Variáveis de decisão: quantidades dos alimentos**

In [17]:
x = model.addVars(len(c_j), lb=f_min, ub=f_max, vtype=GRB.CONTINUOUS, name="x")

#### **Função objetivo: minimizar custo total**

In [18]:
model.setObjective(sum(c_j[j] * x[j] for j in range(len(c_j))), GRB.MINIMIZE)

#### **Restrições: atender ao VDR de cada nutriente**

In [27]:
# Restrições: atender ao VDR de cada nutriente
for i in range(len(r_i)):
    model.addConstr(sum(a_ij[i][j] * x[j] for j in range(len(c_j))) >= r_i[i], name=f"Nutriente_{i}")

#### **Resolver o modelo**

In [ ]:
model.optimize()

In [ ]:
# Resultados
if model.status == GRB.OPTIMAL:
    print("Custo mínimo:", model.objVal)
    print("Quantidades ótimas dos alimentos:")
    for j in range(len(c_j)):
        print(f"{alimentos.iloc[j]['FOOD']}: {x[j].x} g")
else:
    print("Não foi encontrada uma solução ótima.")



---



## **3. Análise de Sensibilidade**

In [28]:
var_data = []
for variavel in model.getVars():
    var_data.append({
        "Variável": variavel.VarName,
        "SAObjLow (Custo Min)": variavel.SAObjLow,
        "SAObjUp (Custo Max)": variavel.SAObjUp,
    })

var_df = pd.DataFrame(var_data)

print("\nAnálise de Sensibilidade - Variáveis:")
print(var_df)

constr_data = []
for restricao in model.getConstrs():
    constr_data.append({
        "Restrição": restricao.ConstrName,
        "SARHSLow (Recurso Min)": restricao.SARHSLow,
        "SARHSUp (Recurso Max)": restricao.SARHSUp,
    })

constr_df = pd.DataFrame(constr_data)

print("\nAnálise de Sensibilidade - Restrições:")
print(constr_df)

var_df.to_csv("sensibilidade_variaveis.csv", index=False)
constr_df.to_csv("sensibilidade_restricoes.csv", index=False)


Análise de Sensibilidade - Variáveis:
   Variável  SAObjLow (Custo Min)  SAObjUp (Custo Max)
0      x[0]              3.895413                  inf
1      x[1]              6.279817                  inf
2      x[2]              5.077064                  inf
3      x[3]              5.477982                  inf
4      x[4]              4.676147                  inf
..      ...                   ...                  ...
79    x[79]              1.112844                  inf
80    x[80]              1.077982                  inf
81    x[81]              1.311009                  inf
82    x[82]              0.506494             2.303457
83    x[83]              1.307339                  inf

[84 rows x 3 columns]

Análise de Sensibilidade - Restrições:
     Restrição  SARHSLow (Recurso Min)  SARHSUp (Recurso Max)
0  Nutriente_0                    -inf             136.697248
1  Nutriente_1               82.170543             296.428571
2  Nutriente_2                    -inf             1

### **Interpretação dos Valores da Análise de Sensibilidade**

#### **Vetor de Custos (Variáveis):**

Os valores **SAObjLow** e **SAObjUp** mostram o quão sensível a solução ótima é às mudanças no custo das variáveis de decisão:

1. **Variáveis com limite superior infinito (`inf`)**:
   - Indicam que o custo dessas variáveis pode aumentar indefinidamente sem alterar a solução ótima.
   - Isso reflete que, na solução atual, essas variáveis têm uma importância menor para o custo total ou que há flexibilidade em sua utilização.

2. **Variáveis com limites mais estreitos**:
   - Indicam maior sensibilidade, ou seja, pequenas mudanças nos custos dessas variáveis podem tornar a solução ótima inválida.
   - Estas variáveis estão mais próximas de serem competitivas com outras alternativas.

**Exemplo:**  
Se uma variável tem **SAObjLow = 5.0** e **SAObjUp = 7.0**, isso significa que, enquanto o custo dessa variável estiver entre 5 e 7, a solução ótima permanecerá a mesma. Se sair desse intervalo, outras variáveis ou combinações podem ser ativadas na solução.

---

#### **Vetor de Recursos (Restrições):**

Os valores **SARHSLow** e **SARHSUp** indicam a faixa de variação do recurso permitido sem alterar a solução ótima:

1. **Restrições com limite inferior negativo infinito (`-inf`)**:
   - Indicam que reduzir o recurso associado não altera a solução, desde que o valor do recurso não ultrapasse o limite superior.
   - Essas restrições são menos críticas e têm maior flexibilidade.

2. **Restrições com limites mais estreitos**:
   - Restrições sensíveis, onde pequenas alterações no recurso disponível podem invalidar a solução ótima.
   - Tais restrições geralmente são "restrições ativas", ou seja, estão no limite de viabilidade na solução.

**Exemplo:**  
Se uma restrição tem **SARHSLow = 100** e **SARHSUp = 200**, isso significa que o recurso disponível pode variar entre 100 e 200 sem alterar a solução. Valores fora desse intervalo podem tornar a solução inviável ou alterar as variáveis ativas.


## **4. Dieta Ótima Vegetariana**

Na tabela de alimentos, existe uma coluna chamada `veg`, que indica se o alimento é vegetariano (`True`) ou não (`False`). Para calcular a dieta ótima vegetariana, filtraremos os dados para considerar apenas os alimentos marcados como vegetarianos.

In [ ]:
# Filtrar apenas os alimentos vegetarianos
alimentos_veg = alimentos[alimentos['veg'] == True]

a_ij = alimentos_veg.iloc[:, 1:len(nutrientes)+1].values.T  # Nutrientes por alimento
c_j = alimentos_veg['cost'].values  # Custos dos alimentos vegetarianos
f_min = alimentos_veg['f_min'].values  # Limites inferiores
f_max = alimentos_veg['f_max'].values  # Limites superiores
r_i = nutrientes['n_min'].values  # VDR dos nutrientes

model = Model("DietaVegetariana")

# Variáveis de decisão: quantidades dos alimentos vegetarianos
x = model.addVars(len(c_j), lb=f_min, ub=f_max, vtype=GRB.CONTINUOUS, name="x")

# Função objetivo
model.setObjective(sum(c_j[j] * x[j] for j in range(len(c_j))), GRB.MINIMIZE)

# Restrições
for i in range(len(r_i)):
    model.addConstr(sum(a_ij[i][j] * x[j] for j in range(len(c_j))) >= r_i[i], name=f"Nutriente_{i}")

# Resultados
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Custo mínimo para dieta vegetariana:", model.objVal)
    print("Quantidades ótimas dos alimentos vegetarianos:")
    for j in range(len(c_j)):
        if x[j].x > 0:  # Exibir apenas alimentos com quantidades positivas
            print(f"{alimentos_veg.iloc[j]['FOOD']}: {x[j].x} g")
else:
    print("Não foi encontrada uma solução ótima.")


## **5. Alimentos únicos**

As variáveis de decisão $ x_j $ passam a ser **binárias** ($ x_j = 0 $ ou $ x_j = 1 $), representando se o alimento $ j $ será consumido ($ 1 $) ou não ($ 0 $).
- Para garantir que os nutrientes sejam atendidos, assumimos que o alimento é consumido **uma única vez na porção padrão** (exemplo: $ 1 $ unidade ou porção definida na tabela).
- O custo total será calculado somando apenas os alimentos selecionados.

In [ ]:
a_ij = alimentos.iloc[:, 1:len(nutrientes)+1].values.T  # Nutrientes por alimento (transposto)
c_j = alimentos['cost'].values  # Custos dos alimentos
r_i = nutrientes['n_min'].values  # VDR dos nutrientes

model = Model("DietaUmaVez")

# Variáveis de decisão: 0 ou 1 (consumir ou não consumir o alimento)
x = model.addVars(len(c_j), vtype=GRB.BINARY, name="x")

# Função objetivo
model.setObjective(sum(c_j[j] * x[j] for j in range(len(c_j))), GRB.MINIMIZE)

# Restrições
for i in range(len(r_i)):
    model.addConstr(sum(a_ij[i][j] * x[j] for j in range(len(c_j))) >= r_i[i], name=f"Nutriente_{i}")

# Resultados
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Custo mínimo para dieta com alimentos únicos:", model.objVal)
    print("Alimentos selecionados:")
    for j in range(len(c_j)):
        if x[j].x > 0.5:  # Exibir apenas os alimentos selecionados
            print(f"{alimentos.iloc[j]['FOOD']}: Consumido 1 vez")
else:
    print("Não foi encontrada uma solução ótima.")


#### *`Observações:`*
- Considerei como se a pessoa pudesse somente comer um alimento inteiro, não considerando que ele possa comer metade de uma maçã, por exemplo. Por isso a variável binária.